In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
!ls

drive  sample_data


In [4]:
!/content/drive/MyDrive

/bin/bash: /content/drive/MyDrive: Is a directory


In [5]:
from google.colab import files

In [6]:
uploaded = files.upload()

Saving crf.py to crf.py
Saving m_resunet.py to m_resunet.py
Saving metrics.py to metrics.py
Saving mobilenetv2_unet.py to mobilenetv2_unet.py
Saving model.py to model.py
Saving resnet50_unet.py to resnet50_unet.py
Saving resunet.py to resunet.py
Saving resunet++_pytorch.py to resunet++_pytorch.py
Saving train.py to train.py
Saving tta.py to tta.py
Saving unet.py to unet.py
Saving utils.py to utils.py


In [7]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from metrics import dice_coef, iou, jacard_coef
from train import load_data

H = 576
W = 576

In [8]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)   ## (1, 256, 256, 3)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x

In [9]:
def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1) ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_path, cat_images)

In [10]:
""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def save_results(image, mask, y_pred, save_image_path):
    ## i - m - y
    line = np.ones((H, 10, 3)) * 512

    """ Mask """
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)

    """ Predicted Mask """
    y_pred = np.expand_dims(y_pred, axis=-1)    ## (512, 512, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)  ## (512, 512, 3)
    y_pred = y_pred * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [11]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("/content/drive/MyDrive/iPSField2/RA-SE-ASPP-Net")

    """ Loading model """
    with CustomObjectScope({'iou': iou,'f1_score': f1_score, 'jacard_coef': jacard_coef,'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("/content/drive/MyDrive/iPS_Field2A/files/RA-SE-ASPP-Net_iPSField2.h5")

    """ Load the dataset """
    test_x = sorted(glob(os.path.join("/content/drive/MyDrive/iPS_Field2A/new_data/", "images", "*")))
    test_y = sorted(glob(os.path.join("/content/drive/MyDrive/iPS_Field2A/new_data/", "masks", "*")))
    print(f"Test: {len(test_x)} - {len(test_y)}")

    """ Evaluation and Prediction """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extract the name """
        name = x.split("/")[-1].split(".")[0]

        """ Reading the image """
        image = cv2.imread(x, cv2.IMREAD_COLOR)
        x = image/255.0
        x = np.expand_dims(x, axis=0)

        """ Reading the mask """
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        y = mask/255.0
        y = y > 0.5
        y = y.astype(np.int32)

        """ Prediction """
        y_pred = model.predict(x)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)

        """ Saving the prediction """
        save_image_path = f"/content/drive/MyDrive/iPSField2/RA-SE-ASPP-Net/{name}.png"
        save_results(image, mask, y_pred, save_image_path)
       

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating the metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ Metrics values """
    score = [s[1:]for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("/content/drive/MyDrive/iPSField2/files/RA-SE-ASPP-Netdata.csv")


Test: 47 - 47


  0%|          | 0/47 [00:00<?, ?it/s]

1/1 [==============================] - 12s 12s/step


  2%|▏         | 1/47 [00:15<12:01, 15.68s/it]

1/1 [==============================] - 0s 25ms/step


...

1/1 [==============================] - 0s 23ms/step


 98%|█████████▊| 46/47 [02:44<00:03,  3.35s/it]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 47/47 [02:47<00:00,  3.57s/it]


Accuracy: 0.98085
F1: 0.96559
Jaccard: 0.93351
Recall: 0.96435
Precision: 0.96685
